In [1]:
import os, glob
from pathlib import Path
import xarray as xr
import pyvista as pv

In [15]:
folder = '/Users/josephko/research/cm1_bulk_ppe/sample_out'
ds_list = []
for f in sorted(glob.glob(os.path.join(folder, 'cm1out_0*'))):
    f_temp = f.split(".",1)[0].rsplit("_", 1)[1]
    f_int = int(f_temp)
    if (f_int >= 10) & (f_int < 50):
        # print(f)
        ds = xr.open_dataset(f)
        variables = ['qi1', 'qi2', 'qi3', 'qi4']
        ds_qi = ds[variables]
        # ds_mean = ds_qi.mean(dim='yh')
        ds_list.append(ds_qi)
    
ds = xr.concat(ds_list, dim = 'time')
ds 

<xarray.Dataset>
Dimensions:  (time: 40, zh: 280, yh: 256, xh: 256)
Coordinates:
  * time     (time) timedelta64[ns] 00:45:00 00:50:00 ... 03:55:00 04:00:00
  * xh       (xh) float32 0.025 0.075 0.125 0.175 ... 12.63 12.68 12.73 12.78
  * yh       (yh) float32 0.025 0.075 0.125 0.175 ... 12.63 12.68 12.73 12.78
  * zh       (zh) float32 0.025 0.075 0.125 0.175 ... 13.83 13.88 13.93 13.98
Data variables:
    qi1      (time, zh, yh, xh) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    qi2      (time, zh, yh, xh) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    qi3      (time, zh, yh, xh) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    qi4      (time, zh, yh, xh) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes: (12/71)
    CM1 version:    cm1r21.0
    Conventions:    CF-1.7
    missing_value:  -999999.9
    x_units:        km
    x_label:        x
    y_units:        km
    ...             ...
    cgt1:           1.875
    cgt2:           -1.25
    cgt3:           0.375
    dgt1:           2.0
    dgt2:           -3.0
    dgt3:           1.0

In [16]:
ds_new = ds.assign(qi=ds['qi1']+ds['qi2']+ds['qi3']+ds['qi4'])
ds_new

<xarray.Dataset>
Dimensions:  (time: 40, zh: 280, yh: 256, xh: 256)
Coordinates:
  * time     (time) timedelta64[ns] 00:45:00 00:50:00 ... 03:55:00 04:00:00
  * xh       (xh) float32 0.025 0.075 0.125 0.175 ... 12.63 12.68 12.73 12.78
  * yh       (yh) float32 0.025 0.075 0.125 0.175 ... 12.63 12.68 12.73 12.78
  * zh       (zh) float32 0.025 0.075 0.125 0.175 ... 13.83 13.88 13.93 13.98
Data variables:
    qi1      (time, zh, yh, xh) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    qi2      (time, zh, yh, xh) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    qi3      (time, zh, yh, xh) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    qi4      (time, zh, yh, xh) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    qi       (time, zh, yh, xh) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes: (12/71)
    CM1 version:    cm1r21.0
    Conventions:    CF-1.7
    missing_value:  -999999.9
    x_units:        km
    x_label:        x
    y_units:        km
    ...             ...
    cgt1:           1.875
    cgt2:           -1.25
    cgt3:           0.375
    dgt1:           2.0
    dgt2:           -3.0
    dgt3:           1.0

In [4]:
# test with single frame 
ds_slice = ds_new.isel(time=0)
da = ds_slice.qi
mesh = da.pyvista.mesh(x='xh', y='yh', z='zh')
mesh.plot(opacity='linear', cmap='binary', volume=True, clim = [0, 0.0002])

Widget(value="<iframe src='http://localhost:63049/index.html?ui=P_0x107ba6710_0&reconnect=auto' style='width: …

In [18]:
### === Create GIF from 10 frames === 
# Create plotter
ds_slice = ds_new.isel(time=0)
da = ds_slice.qi
mesh = da.pyvista.mesh(x='xh', y='yh', z='zh')
plotter = pv.Plotter(notebook=False, off_screen=True, window_size=[1024, 1024])
n_frames = len(ds_new.time)
plotter.add_volume(
    mesh,
    opacity='linear',
    cmap="binary",
    show_scalar_bar=False,
    clim=[0, 0.0002],
)

plotter.open_gif("iceball-3d-viz-test.gif")

# render and don't close
plotter.show(auto_close=False)
plotter.write_frame()

# update each frame
for i in range(1, n_frames):
    print(f'Actors: {plotter.actors}')
    print(f'loop {i}')
    ds_time_i = ds_new.isel(time=i)
    da_qi = ds_time_i.qi
    mesh.point_data['qi'] = da_qi.to_numpy().flatten()
    # print(mesh.active_scalars_info)
    # print('removing actor')
    removed = plotter.clear_actors()
    # print(f'removed? {removed}')
    # print(f'Actors: {plotter.actors}')
    current_scalar = f'qi_{i}'
    actor = plotter.add_volume(
        mesh,
        opacity='linear',
        cmap="binary",
        show_scalar_bar=False,
        clim=[0, 0.0002],
        render=True
    )
    plotter.write_frame()
    # plotter.screenshot(f'screenshot_{i}.png')

plotter.close()

Actors: {'RectilinearGrid(Addr=0x2bc4afe20)': <Volume(0x2bbd56e00) at 0x280e843a0>}
loop 1
ActiveArrayInfoTuple(association=<FieldAssociation.POINT: 0>, name='qi')
removing actor
removed? None
Actors: {}
Actors: {'RectilinearGrid(Addr=0x2bc480860)': <Volume(0x36d8aba00) at 0x29d1fa8c0>}
loop 2
ActiveArrayInfoTuple(association=<FieldAssociation.POINT: 0>, name='qi')
removing actor
removed? None
Actors: {}
Actors: {'RectilinearGrid(Addr=0x2bc4afe20)': <Volume(0x103f18e00) at 0x38df72c20>}
loop 3
ActiveArrayInfoTuple(association=<FieldAssociation.POINT: 0>, name='qi')
removing actor
removed? None
Actors: {}
Actors: {'RectilinearGrid(Addr=0x2bc4cf7c0)': <Volume(0x10d9e1a00) at 0x280e84be0>}
loop 4
ActiveArrayInfoTuple(association=<FieldAssociation.POINT: 0>, name='qi')
removing actor
removed? None
Actors: {}
Actors: {'RectilinearGrid(Addr=0x2c633a370)': <Volume(0x36d8aba00) at 0x16839e020>}
loop 5
ActiveArrayInfoTuple(association=<FieldAssociation.POINT: 0>, name='qi')
removing actor
remov